# Analyze Maricopa County, Arizona  Biodiversity Intactness Index (BII) changes from 2017-2020

Author: Hazel Vaquero

Repository Link: https://github.com/hazelvaq/Maricopa-Arizona-Changes-in-Biodiversity-Intactness-Index-.git 

## About 

In this notebook we analyse the changes observed in Biodiversity Intactness Index (BII) in Maricopa County, Phoenix, Arizona from 2017 to 2020. In 2021, Maricopa County was reported as the fastest growing county in the United States since 2001. It is the 4th largest county in terms of population. Urban development creates pressure on biodiversity and natural areas. This notebook seeks to analyze the impact of the rapid urban expansion in Maricopa County. 

## About the Datasets

In this notebook we use two different datasets:

**1. BII Data** 

`io-biodiversity` collection

**2. Phoenix Subdivison** 

## Final Output

### Import Libraries 

In [6]:
# Import Libraries
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

# Used to access STAC catalogs
from pystac_client import Client
# Used to sign items from the MPC STAC catalog
import planetary_computer

# other libraries for nice outputs
from IPython.display import Image

ModuleNotFoundError: No module named 'pystac_client'

### Import Data

In [2]:
# Data Import: BII data

## Acess Planatery Computer Catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## Catalog Search
# Set temporal range
time_range = "2017/2020"
# Bonding box of Phoenix subdivision
bbox = {
   "type": "Polygon",
    "coordinates": [
        [
            [-112.826843, 32.974108],
            [-111.184387, 32.974108],
            [-111.184387, 33.863574],
            [-112.826843, 33.863574],
            [-112.826843, 32.974108]  # Make sure the first and last coordinates are the same to close the polygon
        ]
    ]
}
# catalog search
search = catalog.search(
    collections=['io-biodiversity'],
    intersects=bbox,
    datetime = time_range
)

# Get items found in catalog search
item = search.item_collection()

# get first item in the catalog search
item = items[0]

In [12]:
# Data Import: Phoenix subdivision
phoenix_counties = gpd.read_file('data/tl_2022_04_cousub.shp')